In [ ]:
!pip install paho-mqtt flask opencv-python numpy requests pyngrok
!pip install paho-mqtt opencv-python numpy requests flask
!pip install pyngrok
!pip install flask opencv-python numpy pyngrok

ERROR: Operation cancelled by user
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/pip/_internal/cli/base_command.py", line 179, in exc_logging_wrapper
    status = run_func(*args)
             ^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/pip/_internal/cli/req_command.py", line 67, in wrapper
    return func(self, options, args)
           ^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/pip/_internal/commands/install.py", line 447, in run
    conflicts = self._determine_conflicts(to_install)
                ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/pip/_internal/commands/install.py", line 578, in _determine_conflicts
    return check_install_conflicts(to_install)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/pip/_internal/operations/check.py", line 101, in check_install_conflicts
    package_set, _ = create_package_

In [ ]:
from pyngrok import ngrok

# Ganti token ngrok (jangan lupa buat akun e ya cuy)
NGROK_AUTH_TOKEN = ""

# Setel
!ngrok authtoken $NGROK_AUTH_TOKEN


Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [ ]:
import cv2
import numpy as np
import base64
import paho.mqtt.client as mqtt
from flask import Flask, request
from pyngrok import ngrok

app = Flask(__name__)

# ========== Setup MQTT ==========
broker = "broker.emqx.io"
mqtt_topic = "la/server/url"

client = mqtt.Client()
client.connect(broker, 1883, 60)

NGROK_AUTH_TOKEN = ""
ngrok.set_auth_token(NGROK_AUTH_TOKEN)

public_url = ngrok.connect(5000).public_url
print("Ngrok Public URL:", public_url)


client.publish(mqtt_topic, public_url)
print(f"URL {public_url} berhasil dikirim ke MQTT!")

# get gambar
@app.route("/", methods=["POST"])
def receive_image():
    try:
        img_data = request.data  # Terima gambar dari ESP32-CAM
        np_arr = np.frombuffer(img_data, np.uint8)
        img = cv2.imdecode(np_arr, cv2.IMREAD_COLOR)

        # saving
        cv2.imwrite("received_image.jpg", img)
        print("Gambar diterima dan disimpan!")

        # (Nantinya, di sini akan ditambahkan Face Recognition)
        return "Gambar diterima!", 200
    except Exception as e:
        print("Error:", e)
        return "Terjadi kesalahan", 500

# Flask
if __name__ == "__main__":
    app.run(host="0.0.0.0", port=5000)


<ipython-input-10-74ed175eaf68>:15: DeprecationWarning: Callback API version 1 is deprecated, update to latest version
  client = mqtt.Client()


Ngrok Public URL: https://0bbf-34-106-240-203.ngrok-free.app
URL https://0bbf-34-106-240-203.ngrok-free.app berhasil dikirim ke MQTT!
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5000
 * Running on http://172.28.0.12:5000
INFO:werkzeug:Press CTRL+C to quit
